# Wideband Statecount and Histogram Data Collection

## Setup

In [ ]:
import os
import csv
import pathlib
import shutil
import numpy as np
from tabulate import tabulate
from PyTango import DeviceProxy

import notebook_tools.histogram_client as HistogramClient
from notebook_tools.misc_helper import get_tango_host

First, get the ns to check and set to be used at the tango host for future steps.

In [ ]:
!kubectl get ns | grep ska-mid-psi

Set these as required, with the namespace pointing to a namespace that has completed a scan. Set VCC to to the boards used by the scan.

In [ ]:
namespace = "ci-ska-mid-psi-1715911277-alexschell"
vccs = [2]
lanes = [1]
timeout_ms = 3000
path = os.getcwd() + "/output-data/" + namespace

To grab the histogram client and access devices, we set the TANGO host environment variable to point to the namespace:

In [ ]:
os.environ["TANGO_HOST"] = get_tango_host(namespace)

In [ ]:
pre_vcc_x = []
pre_vcc_y = []
post_vcc_x_lanes = []
post_vcc_y_lanes = []
state_count_vec_1 = []
state_count_vec_2 = []
power_spectrum_1 = []
power_spectrum_2 = []

## Histogram Data Collection

The following steps collect the histogram data from the respective stages in the signal chain:

### Pre-VCC

To collect the pre-VCC data, step through each of the VCCs, and collect the histogram from each:

In [ ]:
for vcc in vccs:
    device = f"talondx-00{vcc}/histogram/e_pre_vcc"
    client = HistogramClient.HistogramClient(device, timeout_ms)

    # data is an array of [pol_x_data, pol_y_data], or None if success is false.
    success, data = client.capture()
    if success:
        print("Pre-VCC data capture successful, storing")
        pre_vcc_x = data[0]
        pre_vcc_y = data[1]
        print(f"Pol-x Histogram is of dimensions {len(pre_vcc_x)}X{len(pre_vcc_x[0])}")
        print(f"Pol-y Histogram is of dimensions {len(pre_vcc_y)}X{len(pre_vcc_y[0])}")
    else:
        print("Could not get Pre-VCC data!")

To display the plots if desired

In [ ]:
print("Pol X histogram data:")
pol_x_headers = range(len(pre_vcc_x))
print(tabulate(pre_vcc_x, tablefmt="simple", headers=pol_x_headers, showindex="always"))
print("Pol Y histogram data:")
pol_y_headers = range(len(pre_vcc_y))
print(tabulate(pre_vcc_y, tablefmt="simple", headers=pol_y_headers, showindex="always"))

### Post-VCC

To collect the post-vcc histogram data, we check each lane

In [ ]:
for vcc in vccs:
    for lane in lanes:
        device = f"talondx-00{vcc}/histogram/e_post_vcc_{lane}"
        client = HistogramClient.HistogramClient(device, timeout_ms)
        success, data = client.capture()
        if success:
            print(f"Captured data for Board {vcc}, lane {lane}")
            post_vcc_x = data[0]
            post_vcc_x_lanes.append(post_vcc_x)
            post_vcc_y = data[1]
            post_vcc_y_lanes.append(post_vcc_y)
            print(f"Pol-x Histogram is of dimensions {len(post_vcc_x)}X{len(post_vcc_x[0])}")
            print(f"Pol-y Histogram is of dimensions {len(post_vcc_y)}X{len(post_vcc_y[0])}")
        else:
            print(f"Could not capture data for Board {vcc}, lane {lane}")

### Store Histogram Data

Once the histograms are collected, write them to relevant CSV files:

In [ ]:
histogram_folder = path + "/histograms/"
folder = pathlib.Path(histogram_folder)
folder.mkdir(parents=True, exist_ok=True)
if pre_vcc_x.any():
    with open(histogram_folder + "pre_vcc_x.csv", "w") as csvfile:
        writer = csv.writer(csvfile)
        writer.writerows(pre_vcc_x)
        print(f"Wrote histogram data to {histogram_folder}pre_vcc_x.csv")
if pre_vcc_y.any():
    with open(histogram_folder + "pre_vcc_y.csv", "w") as csvfile:
        writer = csv.writer(csvfile)
        writer.writerows(pre_vcc_y)
        print(f"Wrote histogram data to {histogram_folder}pre_vcc_y.csv")

## Statecount Collection

The wideband statecount can also be collected:

In [ ]:
vcc_statecount_holder = []
for vcc in vccs:
    device = f"talondx-00{vcc}/wbstatecount/state_count"
    talon_proxy = DeviceProxy(device)
    talon_proxy.command_inout("state_count_capture")
    state_count_vec_1 = talon_proxy.read_attribute("state_count_vector_1").value
    lst = list(state_count_vec_1)
    lst.insert(0, "State Count Vector 1")
    state_count_vec_1 = np.asarray(lst)

    state_count_vec_2 = talon_proxy.read_attribute("state_count_vector_2").value
    lst = list(state_count_vec_2)
    lst.insert(0, "State Count Vector 2")
    state_count_vec_2 = np.asarray(lst)

    power_spectrum_1 = talon_proxy.read_attribute("psd_vector_1").value
    lst = list(power_spectrum_1)
    lst.insert(0, "Power Spectrum 1")
    power_spectrum_1 = np.asarray(lst)

    power_spectrum_2 = talon_proxy.read_attribute("psd_vector_2").value
    lst = list(power_spectrum_2)
    lst.insert(0, "Power Spectrum 2")
    power_spectrum_2 = np.asarray(lst)

    vcc_statecount_holder.append(
        [state_count_vec_1, state_count_vec_2, power_spectrum_1, power_spectrum_2]
    )

Format to make these a bit easier to read when output to CSV:

In [ ]:
print(tabulate(vcc_statecount_holder[0], tablefmt="simple"))

In [ ]:
statecount_folder = path + "/statecounts/"
folder = pathlib.Path(statecount_folder)
folder.mkdir(parents=True, exist_ok=True)
i = 0
for vcc in vccs:
    with open(statecount_folder + "vcc-" + str(vcc) + "_statecounts.csv", "w") as csvfile:
        writer = csv.writer(csvfile)
        writer.writerows(vcc_statecount_holder[i])
    i += 1

## Package Data

If desired, zip the archive for easier downloading/sharing.

In [ ]:
shutil.make_archive(namespace, "zip", path)